In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as mcm
import matplotlib.gridspec as gridspec
from matplotlib.ticker import PercentFormatter
from matplotlib.colors import LogNorm
from epiweeks import Week
from mpl_toolkits.axes_grid1 import make_axes_locatable
import baltic as bt
from datetime import datetime, timedelta
from matplotlib.patches import Circle
from matplotlib.lines import Line2D
import numpy as np
from utils import setup_plotting_standards, timeseries_formatting

setup_plotting_standards()

In [2]:
# Output from MatUtils
intros = pd.read_csv( "../data/2022-04-26_cog_introduce.tsv", sep='\t' )
intros.head(50)

,sample,introduction_node,introduction_rank,growth_score,earliest_date,latest_date,cluster_size,cluster_span,intro_confidence,parent_confidence,distance,origin_gap,region,origins,origins_confidence,mutation_path
0,EPI_ISL_5329368,Seattle_node_9664,1,0.201452,2021-Sep-22,2021-Oct-22,39,19,1.0,0.000000,0,4,Seattle,Santa Clara,1.000000,C12714T<
1,EPI_ISL_5329343,Seattle_node_9664,1,0.201452,2021-Sep-22,2021-Oct-22,39,19,1.0,0.000000,0,4,Seattle,Santa Clara,1.000000,C12714T<
2,EPI_ISL_5329342,Seattle_node_9664,1,0.201452,2021-Sep-22,2021-Oct-22,39,19,1.0,0.000000,0,4,Seattle,Santa Clara,1.000000,C12714T<
3,EPI_ISL_5329341,Seattle_node_9664,1,0.201452,2021-Sep-22,2021-Oct-22,39,19,1.0,0.000000,0,4,Seattle,Santa Clara,1.000000,C12714T<
4,EPI_ISL_5329340,Seattle_node_9664,1,0.201452,2021-Sep-22,2021-Oct-22,39,19,1.0,0.000000,0,4,Seattle,Santa Clara,1.000000,C12714T<
5,EPI_ISL_5329311,Seattle_node_9664,1,0.201452,2021-Sep-22,2021-Oct-22,39,19,1.0,0.000000,0,4,Seattle,Santa Clara,1.000000,C12714T<
6,EPI_ISL_5329173,Seattle_node_9664,1,0.201452,2021-Sep-22,2021-Oct-22,39,19,1.0,0.000000,0,4,Seattle,Santa Clara,1.000000,C12714T<
7,EPI_ISL_5903226,Seattle_node_9664,1,0.201452,2021-Sep-22,2021-Oct-22,39,19,1.0,0.000000,1,4,Seattle,Santa Clara,1.000000,C12714T<
8,EPI_ISL_5329353,Seattle_node_9664,1,0.201452,2021-Sep-22,2021-Oct-22,39,19,1.0,0.000000,1,4,Seattle,Santa Clara,1.000000,C12714T<
9,EPI_ISL_5329171,Seattle_node_9664,1,0.201452,2021-Sep-22,2021-Oct-22,39,19,1.0,0.000000,0,4,Seattle,Santa Clara,1.000000,C12714T<


In [33]:
region_counts = intros["region"].value_counts().sort_index()
region_counts

Baja California          781
Baja California Sur      146
Bangladesh               602
British Columbia       22705
Chihuahua                273
Connecticut            10775
Denmark1               65876
Denmark2               21435
El Paso                  123
Germany1               30894
Germany2                6964
Imperial                 207
India                   3322
Ireland                 9439
Los Angeles             4568
Malaysia                1263
Maricopa                6705
Mexico City             2378
New South Wales        13644
New York                1842
Ontario                 5040
Other                     21
San Diego              10850
Santa Clara             3925
Seattle                 1640
Singapore               8088
Sonora                   256
Sweden                  8191
Switzerland             1978
Victoria                1955
Yuma                     229
Name: region, dtype: int64

In [13]:
len( intros["introduction_node"].unique() )

29189

In [22]:
intros = pd.read_csv( "../data/2022-04-26_cog_introduce.tsv", sep='\t' )
test = intros.drop( columns=["sample", "distance", "intro_confidence"] ).drop_duplicates()
test = test.pivot_table( columns="region", index="origins", values="introduction_node", aggfunc="count", fill_value=0 )
abs_count = test.melt( ignore_index=False, value_name="intros" )

test = test.apply( lambda x: x / x.sum() )
prop = test.melt( ignore_index=False, value_name="proportion" )

output = abs_count.merge( prop, on=["origins", "region"] )
output["file"] = "actual.1"
output.head()

,region,intros,proportion,file
origins,,,,
Baja California,Baja California,0,0.000000,actual.1
Baja California Sur,Baja California,5,0.009862,actual.1
Bangladesh,Baja California,0,0.000000,actual.1
British Columbia,Baja California,2,0.003945,actual.1
Chihuahua,Baja California,7,0.013807,actual.1


In [23]:
import os

In [19]:
test.apply( lambda x: x / x.sum() )

region,Baja California,Baja California Sur,Bangladesh,British Columbia,Chihuahua,Connecticut,Denmark1,Denmark2,El Paso,Germany1,...,Other,San Diego,Santa Clara,Seattle,Singapore,Sonora,Sweden,Switzerland,Victoria,Yuma
origins,,,,,,,,,,,,,,,,,,,,,
Baja California,0.000000,0.068966,0.000000,0.004573,0.043165,0.004869,0.000000,0.000498,0.000000,0.000537,...,0.0,0.044452,0.010937,0.005006,0.000000,0.053254,0.001355,0.000000,0.00000,0.01875
Baja California Sur,0.009862,0.000000,0.000000,0.001524,0.028777,0.001217,0.000344,0.000000,0.000000,0.000269,...,0.0,0.004698,0.005469,0.002503,0.000000,0.035503,0.000000,0.000000,0.00000,0.00000
Bangladesh,0.000000,0.000000,0.000000,0.001524,0.000000,0.003043,0.002405,0.000498,0.000000,0.002955,...,0.0,0.002530,0.000781,0.000000,0.061181,0.000000,0.002710,0.001379,0.04375,0.00000
British Columbia,0.003945,0.000000,0.016260,0.000000,0.000000,0.026172,0.009966,0.005979,0.000000,0.008595,...,0.0,0.017347,0.016406,0.032541,0.040084,0.000000,0.006775,0.009655,0.03125,0.00000
Chihuahua,0.013807,0.011494,0.000000,0.001524,0.000000,0.004869,0.000000,0.000000,0.038961,0.000269,...,0.0,0.005421,0.001563,0.002503,0.000000,0.011834,0.000000,0.000000,0.00000,0.01250
Connecticut,0.025641,0.000000,0.016260,0.092988,0.071942,0.000000,0.018900,0.010463,0.025974,0.016116,...,0.0,0.116372,0.136719,0.131414,0.037975,0.005917,0.013550,0.006897,0.01250,0.05000
Denmark1,0.007890,0.000000,0.113821,0.071646,0.043165,0.045040,0.000000,0.457897,0.025974,0.284985,...,0.0,0.017347,0.017188,0.022528,0.113924,0.000000,0.336721,0.168276,0.13750,0.01250
Denmark2,0.000000,0.000000,0.016260,0.028963,0.007194,0.010956,0.209622,0.000000,0.000000,0.128122,...,0.0,0.005421,0.008594,0.005006,0.040084,0.005917,0.129404,0.062069,0.03125,0.00000
El Paso,0.003945,0.000000,0.000000,0.001524,0.007194,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.001446,0.003125,0.003755,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000


In [26]:
os.path.splitext( os.path.split( "temp/collapsed_results/{kind}.{num}.csv" )[-1] )[0]

'{kind}.{num}'